<a href="https://colab.research.google.com/github/kfcislif3/NLPLabs/blob/main/Collab_filter_Casino_No_Ex_010521.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Implementation of Collaborative Filtering using Matrix Factorisation on Casino Dataset (from Kindred)
---
---
1st implementation without post-hoc explanation

2nd implementation will be with post-hoc explanation

---
Structure of the Files is as follows:





In [4]:
##########################
## Google Drive Imports ##
##########################

from google.colab import drive
drive.mount('/content/drive') # Outputs will be saved in your google drive

Mounted at /content/drive


In [5]:
!pip install implicit
!pip install ml_metrics

     |████████████████████████████████| 1.1MB 5.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406327 sha256=4b1301ad1672d4284c410e718e0a59e9d00679494199683b42898de538676575
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit
  Created wheel for ml-metrics: filename=ml_metrics-0.1.4-cp37-none-any.whl size=7850 sha256=172226e0964ed8292f178eacb20ada64ccb860137a3a9c6cff6295c6e71e9c27
  Stored in directory: /root/.cache/pip/wheels/b3/61/2d/776be7b8a4f14c5db48c8e5451451cabc58dc6aa7ee3801163
Successfully built ml-metrics


In [72]:
## Checking GPU type
import torch
torch.cuda.is_available()
torch.cuda.get_device_name()

'Tesla V100-SXM2-16GB'

In [6]:
#####################
## Library Imports ##
#####################

## Data Processing Libraries
import pandas as pd
import numpy as np
import datetime
import scipy.sparse as sparse
import csv

# Misc Libraries
import os

## ML Libraries
from sklearn.model_selection import train_test_split
import implicit
import ast
import ml_metrics

## Key Parameters
random_state = 1


In [7]:
## Data Loading
data_dir = '/content/drive/My Drive/Colab/Indivi_Proj/student_project_recommendations.csv' ## 14 days worth of daily casino login data

data = pd.read_csv(data_dir)
print(data.shape)

(2672042, 6)


In [9]:
## Adding column of Date in yyyy/mm/dd hh:mm:ss format from timestamp variable
df = pd.DataFrame(np.round(data['timestamp']/1000, 0))
df['date'] = pd.to_datetime(df['timestamp'], unit='s') # New column with date in yyyy/mm/dd hh:mm:ss format
df = df.drop(['timestamp'], axis=1)
data_withdate = pd.concat([data, df], axis=1) # New dataframe with new column with date in yyyy/mm/dd hh:mm:ss format
data_withdate

,timestamp,provider,channel,country,customer_id,game_id,date
0,1614695395942,CasinoNyx,MOBILE,NO,109208,301a3c173bb7dcf50fc1be952422149aa222f308,2021-03-02 14:29:56
1,1614699098268,CasinoNyx,MOBILE,NO,109208,301a3c173bb7dcf50fc1be952422149aa222f308,2021-03-02 15:31:38
2,1614585331359,CasinoNyx,MOBILE,SE,43126,fdb20ffdaf3bff257cd0ed1a6fd25809612699a0,2021-03-01 07:55:31
3,1615406190976,CasinoNyx,MOBILE,SE,43126,fdb20ffdaf3bff257cd0ed1a6fd25809612699a0,2021-03-10 19:56:31
4,1614800197189,RedTigerGaming,MOBILE,SE,43126,ce47e9dd304d2ca7393766b4d4f39a94aab726b1,2021-03-03 19:36:37
...,...,...,...,...,...,...,...
2672037,1615748719950,LiveCasino.Evolution,WEB,NL,21251,42e686bc2fb9ad31b1fc9d37b70c5a97a6b258a4,2021-03-14 19:05:20
2672038,1615760162659,LiveCasino.Evolution,MOBILE,HU,187158,045b5fdac129b678256b73832ec10666982fc25f,2021-03-14 22:16:03
2672039,1615760162659,LiveCasino.Evolution,MOBILE,HU,187158,b374a0282fa36b5829555203c4683e5e2edb4c22,2021-03-14 22:16:03
2672040,1615760162659,LiveCasino.Evolution,MOBILE,HU,187158,30f252f9da6f81dfa22e2f2fe5920dae15c4c556,2021-03-14 22:16:03


In [10]:
###############################
## Exploratory data analysis ##
###############################
print('Number of timestamps:', len(data_withdate.timestamp.unique()))
print('Number of providers:', len(data_withdate.provider.unique()))
print('Number of channels:', len(data_withdate.channel.unique()))
print('Number of countries:', len(data_withdate.country.unique()))
print('Number of Customers:', len(data_withdate.customer_id.unique()))
print('Number of Games:', len(data_withdate.game_id.unique()))

Number of timestamps: 1284145
Number of providers: 23
Number of channels: 2
Number of countries: 59
Number of Customers: 194480
Number of Games: 2793


In [11]:
## Exploratory data analysis for distribution in number of logins by days
## Maybe can split my countries to have a look. 
## Does it make sense to have different recommendation models for different countries?

## TODO MAYBE TO LOOK AT DISTRIBUTION BY HOURS IN THE DAY
## however need to consider data is across different timezones. so may need to normalise for that.

df['date_day'] = pd.to_datetime(data_withdate['date']).dt.date
print('Number of Games played on different dates :', df.date_day.value_counts())

Number of Games played on different dates : 2021-03-12    213315
2021-03-05    210334
2021-03-06    200417
2021-03-13    198705
2021-03-01    193340
2021-03-11    190689
2021-03-10    190491
2021-03-04    187685
2021-03-09    185649
2021-03-02    184780
2021-03-03    181816
2021-03-08    178029
2021-03-07    177995
2021-03-14    163799
2021-02-28     14998
Name: date_day, dtype: int64


In [37]:
##########################################
## To Do MORE EXPLORATORY DATA ANALYSIS ##
##########################################

## TODO INCLUDE PLOTS FOR EXPLORATORY DATA ANALYSIS


In [38]:
#########################
## Data Pre-Processing ##
#########################

In [13]:
print('Number of missing values:')
check_null = data_withdate.isnull().sum()
check_null
## shows 14 missing values for country.
## no issue as will only be looking at user_id and item_id

Number of missing values:


timestamp       0
provider        0
channel         0
country        14
customer_id     0
game_id         0
date            0
dtype: int64

In [14]:
## Change data type to categorical data for Customer ID and Game ID
data_withdate_customer = data_withdate['customer_id'].astype('category').cat.codes
data_withdate_customer = data_withdate_customer.rename('customer_id_cat')
data_withdate_game_id = data_withdate['game_id'].astype('category').cat.codes
data_withdate_game_id = data_withdate_game_id.rename('game_id_cat')

In [15]:
## Merge Data Types
data_withdate = pd.concat([data_withdate, data_withdate_customer, data_withdate_game_id], axis=1) # New dataframe with new column with date in yyyy/mm/dd hh:mm:ss format

## Adding game_id_count for creating a pivot table of number of games played for each game for each customer
data_withdate['game_id_count'] = 1

## Check dataset
data_withdate

,timestamp,provider,channel,country,customer_id,game_id,date,customer_id_cat,game_id_cat,game_id_count
0,1614695395942,CasinoNyx,MOBILE,NO,109208,301a3c173bb7dcf50fc1be952422149aa222f308,2021-03-02 14:29:56,109207,518,1
1,1614699098268,CasinoNyx,MOBILE,NO,109208,301a3c173bb7dcf50fc1be952422149aa222f308,2021-03-02 15:31:38,109207,518,1
2,1614585331359,CasinoNyx,MOBILE,SE,43126,fdb20ffdaf3bff257cd0ed1a6fd25809612699a0,2021-03-01 07:55:31,43125,2771,1
3,1615406190976,CasinoNyx,MOBILE,SE,43126,fdb20ffdaf3bff257cd0ed1a6fd25809612699a0,2021-03-10 19:56:31,43125,2771,1
4,1614800197189,RedTigerGaming,MOBILE,SE,43126,ce47e9dd304d2ca7393766b4d4f39a94aab726b1,2021-03-03 19:36:37,43125,2241,1
...,...,...,...,...,...,...,...,...,...,...
2672037,1615748719950,LiveCasino.Evolution,WEB,NL,21251,42e686bc2fb9ad31b1fc9d37b70c5a97a6b258a4,2021-03-14 19:05:20,21250,714,1
2672038,1615760162659,LiveCasino.Evolution,MOBILE,HU,187158,045b5fdac129b678256b73832ec10666982fc25f,2021-03-14 22:16:03,187157,65,1
2672039,1615760162659,LiveCasino.Evolution,MOBILE,HU,187158,b374a0282fa36b5829555203c4683e5e2edb4c22,2021-03-14 22:16:03,187157,1962,1
2672040,1615760162659,LiveCasino.Evolution,MOBILE,HU,187158,30f252f9da6f81dfa22e2f2fe5920dae15c4c556,2021-03-14 22:16:03,187157,524,1


In [61]:
############################
## Train/ Val/ Test Split ##
############################

## CREATING A SMALLER DATASET FOR TESTING PURPOSES... TO DELETE THIS AFTER!!!
#train, test = train_test_split(data_withdate, test_size = 0.9, random_state = random_state, shuffle = True)

## Creating training, test, validation splits with Cross Validation
train, test = train_test_split(data_withdate, test_size = 0.2, random_state = random_state, shuffle = True) ## TO DO CHANGE SHUFFLE=TRUE FOR FIRST TRAIN_TEST_SPLIT
train, cross_val = train_test_split(train, test_size = 0.25, random_state = random_state, shuffle = False) 
print('Splitted dataset into train set, cross validation set and test set')
print('Train shape:', train.shape)
print('Cross_val shape:',cross_val.shape)
print('Test shape:', test.shape)


Splitted dataset into train set, cross validation set and test set
Train shape: (1603224, 10)
Cross_val shape: (534409, 10)
Test shape: (534409, 10)


In [62]:
########################
## Recommender System ##
########################
'''
No explicit ratings in the casino  dataset -> need to use implicit feedback from customers. 
Game_ID count is considered as implicit feedback from customers, game_id counts will represents customers's preference or confidence for a specific casino game.

Use implicit Alternating Least Squares (ALS) as a recommendation model. 
Hence, need two csr matrix, user_items (csr_matrix) – Sparse matrix containing the liked items for the user and item_users (csr_matrix)
 – Matrix of confidences for the liked items. 
 This matrix should be a csr_matrix where the rows of the matrix are the item, 
 the columns are the users that liked that item, and the value is the confidence that the user liked the item.
'''

"\nNo explicit ratings in the casino  dataset -> need to use implicit feedback from customers. \nGame_ID count is considered as implicit feedback from customers, game_id counts will represents customers's preference or confidence for a specific casino game.\n\nUse implicit Alternating Least Squares (ALS) as a recommendation model. \nHence, need two csr matrix, user_items (csr_matrix) – Sparse matrix containing the liked items for the user and item_users (csr_matrix)\n – Matrix of confidences for the liked items. \n This matrix should be a csr_matrix where the rows of the matrix are the item, \n the columns are the users that liked that item, and the value is the confidence that the user liked the item.\n"

In [63]:
customer_game = sparse.csr_matrix((train['game_id_count'].astype(float),(train['customer_id_cat'], train['game_id_cat'])))
game_customer = sparse.csr_matrix((train['game_id_count'].astype(float),(train['game_id_cat'], train['customer_id_cat'])))
print(game_customer)  

  (0, 2969)	7.0
  (0, 4570)	1.0
  (0, 5232)	1.0
  (0, 9104)	1.0
  (0, 9333)	1.0
  (0, 9481)	19.0
  (0, 9673)	2.0
  (0, 12896)	3.0
  (0, 17439)	13.0
  (0, 20541)	3.0
  (0, 20840)	1.0
  (0, 22022)	3.0
  (0, 23442)	2.0
  (0, 24620)	1.0
  (0, 24892)	2.0
  (0, 25403)	1.0
  (0, 25677)	1.0
  (0, 27380)	2.0
  (0, 28552)	1.0
  (0, 28955)	1.0
  (0, 29264)	10.0
  (0, 30022)	1.0
  (0, 30838)	2.0
  (0, 31419)	2.0
  (0, 33458)	1.0
  :	:
  (2791, 168821)	1.0
  (2791, 169999)	1.0
  (2791, 170135)	1.0
  (2791, 170319)	8.0
  (2791, 175034)	1.0
  (2791, 175106)	10.0
  (2791, 176798)	1.0
  (2791, 177464)	5.0
  (2791, 178183)	1.0
  (2791, 181905)	6.0
  (2791, 182298)	1.0
  (2791, 183198)	1.0
  (2791, 185430)	2.0
  (2791, 186634)	1.0
  (2791, 186989)	1.0
  (2791, 187592)	1.0
  (2791, 189014)	2.0
  (2791, 191523)	1.0
  (2791, 194046)	1.0
  (2791, 194247)	4.0
  (2792, 44788)	3.0
  (2792, 46285)	7.0
  (2792, 109866)	8.0
  (2792, 149560)	2.0
  (2792, 182148)	1.0


In [73]:
## Model Training

os.environ['MKL_NUM_THREADS'] = '1' #To avoid multithreading.
os.environ['OPENBLAS_NUM_THREADS'] = '1'

model = implicit.als.AlternatingLeastSquares(factors = 512, iterations = 100)
''''Parameters: (factors=100, regularization=0.01, dtype=<type 'numpy.float32'>, use_native=True, use_cg=True, 
use_gpu=False, iterations=15, calculate_training_loss=False, num_threads=0)'''
alpha = 40
train_conf = (game_customer*alpha).astype('double')
model.fit(train_conf)

In [ ]:
## Model Evaluation vs. Train

'''
Model evaluation
Comparision of train, cros_val and test scores. Used mapk as evaluation matric. 
mapk considers the order of the predicted items so recommendation accuracy can be obtained easily.
'''

N = 5 ## N is the number of CASINO GAME recommendations created by the recommedation model for each user/ CUSTOMER

fields = 'customer_id_cat', 'game_list'
filename = 'rec_train.csv'
with open (filename, 'a', newline = '') as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    customer_id_game = train['customer_id_cat'].values.tolist()
    for customer in customer_id_game:
        scores = []
        games =[]
        results = []
        results.append(customer)
        recommendations = model.recommend(customer, customer_game, N = N)
        for game in recommendations:
            ids, score = game
            scores.append(score)
            games.append(ids)
        results.append(games)
        writer.writerow(results)


In [ ]:
predicted = pd.read_csv('rec_train.csv')
predicted = predicted['game_list']

#predicted = [ast.literal_eval(a) for a in predicted] ## ERROR CAUSED. ISSUE NOT FIXED >> ERROR CODE ValueError: malformed node or string: <_ast.Name object at 0x7fc20526b750>
actual = train['game_id_cat']

actual = np.array(actual).reshape(1603224,1) ## Train shape

## MAPK USED AS EVALUATION METRIC FOR RECOMMENDATION MODEL 
## https://towardsdatascience.com/evaluation-metrics-for-recommender-systems-df56c6611093
score = ml_metrics.mapk(actual, predicted, 5) 
print('Mean avg. precision at k for train set:','{:.8f}'.format(score))

In [ ]:
## Model Evaluation vs. Non Held-out Cross Validation set

N = 5

fields = 'customer_id_cat', 'game_list'
filename = 'rec_cross.csv'
with open (filename, 'a', newline = '') as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    customer_id_game = cross_val['customer_id_cat'].values.tolist()
    for customer in customer_id_game:
        scores = []
        games =[]
        results = []
        results.append(customer)
        recommendations = model.recommend(customer, customer_game, N = N)
        for game in recommendations:
            ids, score = game
            scores.append(score)
            games.append(ids)
        results.append(games)
        writer.writerow(results)


In [ ]:
predicted_c = pd.read_csv('rec_cross.csv')
predicted_c = predicted_c['game_list']

#predicted_c = [ast.literal_eval(a) for a in predicted_c]
actual_c = cross_val['game_id_cat']

actual_c = np.array(actual_c).reshape(534409,1) ## Cross Val shape

score_c = ml_metrics.mapk(actual_c, predicted_c, 5) 
print('Mean avg. precision at k for cross validation set:','{:.8f}'.format(score))

In [ ]:
## Model Evaluation vs. Held-out Test set

N = 5

fields = 'customer_id_cat', 'game_list'
filename = 'rec_test.csv'
with open (filename, 'a', newline = '') as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    customer_id_game = test['customer_id_cat'].values.tolist()
    for customer in customer_id_game:
        scores = []
        games =[]
        results = []
        results.append(customer)
        recommendations = model.recommend(customer, customer_game, N = N)
        for game in recommendations:
            ids, score = game
            scores.append(score)
            games.append(ids)
        results.append(games)
        writer.writerow(results)

In [ ]:
predicted_t = pd.read_csv('rec_test.csv')
predicted_t = predicted_t['game_list']

#predicted_t = [ast.literal_eval(a) for a in predicted_t]
actual_t = test['game_id_cat']

actual_t = np.array(actual_t).reshape(534409,1) ## Test shape

score_t = ml_metrics.mapk(actual_t, predicted_t, 5) 
print('Mean avg. precision at k for test set:','{:.8f}'.format(score))